In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompt_values import ChatPromptValue

from langchain_core.messages.base import BaseMessage

template: ChatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}",
        ),
        MessagesPlaceholder("history", n_messages=2),
        ("human", "{text}"),
    ]
)

value: ChatPromptValue = template.format_prompt(
    input_language="English",
    output_language="French",
    history=[("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
    text="I love programming.",
)

messages: list[BaseMessage] = value.to_messages()
for m in messages:
    print(type(m))
    print(m.get_lc_namespace())
    print(m)


<class 'langchain_core.messages.system.SystemMessage'>
['langchain', 'schema', 'messages']
content='You are a helpful assistant that translates English to French' additional_kwargs={} response_metadata={}
<class 'langchain_core.messages.human.HumanMessage'>
['langchain', 'schema', 'messages']
content="what's 5 + 2" additional_kwargs={} response_metadata={}
<class 'langchain_core.messages.ai.AIMessage'>
['langchain', 'schema', 'messages']
content='5 + 2 is 7' additional_kwargs={} response_metadata={}
<class 'langchain_core.messages.human.HumanMessage'>
['langchain', 'schema', 'messages']
content='I love programming.' additional_kwargs={} response_metadata={}


In [5]:
import getpass

password = getpass.getpass("hello")
print(password)

ss


In [19]:
import torch
from abc import ABC
from typing import List, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.models.qwen2.modeling_qwen2 import Qwen2ForCausalLM
from transformers.models.qwen2.tokenization_qwen2_fast import (
    Qwen2TokenizerFast,
)


model_name = "/root/xiatian/models/Qwen2.5-7B-Instruct"
model: Qwen2ForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto", device_map="cpu"
)

tokenizer: Qwen2TokenizerFast = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda:7")
model = model.to(device)


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.03it/s]


In [3]:
print(model.device)

cuda:0


In [9]:
model.half()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [35]:
role = "你是校对助手，负责解决文字中存在的错别字、文字差错、逻辑错误、数字计算错误以及不符合常识的错误，并给出正确答案。请先进行仔细分析，给出推理步骤，遇到数字，则调用计算器进行计算以便验证。分析完毕后，在最后列出存在的问题，然后在最后一行输出修改后的完整句子，如果没有错误，则在最后一行输出：句子无错误"
role = "你是校对助手，负责解决文字中存在的错别字、文字差错、逻辑错误、数字计算错误以及不符合常识的错误，并给出正确答案。请先进行仔细分析，给出推理步骤，遇到数字，则调用计算器进行计算以便验证。在修改结果时，优先选择拼音接近或者字形接近的字词。分析完毕后，列出存在的问题。最后，如果句子没有问题，则输出：句子无错误，否则输出修改后的完整句子。"
role = "你是校对助手，解决文字中存在的错别字词的误用问题，修改时选择拼音或字形接近的字词，思考后列出问题，再输出修改后句子。"
prompt = "他一年用掉了三万多把扫帚打扫卫生。"
prompt = "据估计，2019年北京市垃圾处理总吨数将突破1000万吨，处理1吨垃圾的费用约为300元/吨，处理这些垃圾的总费用就高达3亿元。"
prompt = "青铜峡河段长约8.2公里，宽50100米，两侧的崖壁高30米以上，具有典型的北方粗犷雄渾特色，被称为“黄河上游第一峡谷”。"
prompt = """黄河全长约5644公里，流域面积约75万公里。"""
prompt = """5.下列四组词语中，没有错别字的一组是：
A.羁绊    振振有词 执著.     B.  老奸巨猾  惟美主义 恰如其分
C.勘误  绝不放弃  消声匿迹    D.流蹿  性格不和  进退维谷
"""
prompt = """9.下列典籍中，属于“四书五经”中“四书”的是：
A.《论语》
B.《中庸》
C.《周易》
D.《孟子》"""
prompt = "纠正错别字：既要坚持‘内容为王’，也要加强‘渠道致胜’"
# prompt = "惟美主义"
# prompt = "习平书记出席了会议。"
prompt = "中国城墙的各种物质形态经过数百年的发展、传承和演进，到明清时代已珍于高度成熟，故宫城墙既是明清官式城墙类建筑的典范。故宫城墙始建于明永乐四年（1406年），建成于永乐十八年（1420年），是我国现存规模最大、保存最完整的皇家宫殿城墙，在中国筑城史上占据及其重要的地位。故宫城墙已经列入世界自然遗产名录。"
messages = [
    {
        "role": "system",
        "content": role,
    },
    {"role": "user", "content": prompt},
]

text: str = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer(
    [text], return_tensors="pt", max_length=512, truncation=True
)
model_inputs = model_inputs.to(model.device)
from transformers import GenerationConfig

config = GenerationConfig()
config.temperature = 0
generated_ids = model.generate(
    **model_inputs, generation_config=config, max_new_tokens=512
)

# print(tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0])

# 生成的id删除前面和输入相同的id
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

generated_texts = tokenizer.batch_decode(
    generated_ids, skip_special_tokens=True
)
print(prompt)
print(generated_texts[0])

问题在于“珍于”应为“臻于”，“自然”应为“文化遗产”。

修改后的句子：
中国城墙的各种物质形态经过数百年的发展、传承和演进，到明清时代已臻于高度成熟，故宫城墙既是明清官式城墙类建筑的典范。故宫城墙始建于明永乐四年（1406年），建成于永乐十八年（1420年），是我国现存规模最大、保存最完整的皇家宫殿城墙，在中国筑城史上占据极其重要的地位。故宫城墙已经列入世界文化遗产名录。
